In [ ]:
import scattertext as st
from scattertext.external import phrasemachine
import spacy
import pandas as pd
from pprint import pprint
import spacy
from scattertext import SampleCorpora, PhraseMachinePhrases, dense_rank, RankDifference, AssociationCompactor, produce_scattertext_explorer
from scattertext.CorpusFromPandas import CorpusFromPandas

annotations_df = pd.read_csv('ps_dataset1_cleaned.csv')
#print(annotations_df.iloc[0])

nlp = spacy.load("en_core_web_sm")

corpus = st.CorpusFromPandas(annotations_df, 
                            category_col='category',
                            text_col='annotation_text', 
                            nlp=nlp).build()

#print(list(corpus.get_scaled_f_scores_vs_background().index[:10]))

In [40]:
#Class
term_freq_df = corpus.get_term_freq_df()

In [48]:
terms = term_freq_df.iloc[0]

In [49]:
terms

Gender freq       65
None freq         38
Class freq        36
Pandemics freq    12
Sexuality freq     3
Religion freq      4
Race freq          5
Name: hermione, dtype: int32

In [37]:
term_freq_df.head()

,Gender freq,None freq,Class freq,Pandemics freq,Sexuality freq,Religion freq,Race freq
term,,,,,,,
hermione,65,38,36,12,3,4,5
providing,1,0,1,0,0,1,0
aid,1,0,1,0,0,0,0
to,219,185,235,108,23,21,24
her,96,34,40,5,8,1,7


In [6]:
annotations = annotations_df["annotation_text"][0]

In [7]:
# split into words
from nltk.tokenize import word_tokenize
annotation_tokens = word_tokenize(annotations)
print(annotation_tokens[:100])

['Hermione', 'providing', 'aid', 'to', 'her', 'King', 'and', 'husband', ',', 'almost', 'used', 'as', 'an', 'accessory', '.']


In [8]:
# remove all tokens that are not alphabetic
words = [word for word in annotation_tokens if word.isalpha()]
print(words[:100])

['Hermione', 'providing', 'aid', 'to', 'her', 'King', 'and', 'husband', 'almost', 'used', 'as', 'an', 'accessory']


In [9]:
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))
print(stop_words)

{"mightn't", 'few', 'where', 'who', 'have', "that'll", 'myself', 'very', "mustn't", 'there', 'should', "you'd", 'ourselves', "haven't", 'a', 'only', 're', 'couldn', 'hasn', 'theirs', 'out', 'no', "won't", 'them', 'most', 'then', 'ours', 'you', 'doesn', 'doing', 'against', 'wouldn', 'so', "isn't", 'of', 'all', 'because', 'being', 'those', 'having', "needn't", 'each', 'over', 'd', 'needn', 'themselves', 'our', 'isn', 'aren', 'with', 'my', 'o', "didn't", "she's", 'am', 'they', 'between', 'and', 'than', 'haven', 'more', 'were', 'your', 'whom', 'nor', "hadn't", "wasn't", 'll', 'before', 'he', "aren't", 'been', 'off', 'herself', 'was', 'yours', 'won', 'weren', 'him', 'me', "doesn't", "couldn't", 'himself', "shouldn't", 'below', 'any', 'ain', 'what', 'but', 'during', 'yourselves', 'not', 'wasn', 'too', 'from', 'now', 'into', "you're", "should've", 'hers', 'hadn', 'shouldn', "weren't", "you'll", 'this', 'some', 'above', 'both', 'own', 'i', 'mightn', 'here', 'are', 'didn', 'mustn', 'in', 'the',

In [10]:
# convert to lower case
annotation_tokens = [w.lower() for w in annotation_tokens]
# remove punctuation from each word
import string
table = str.maketrans('', '', string.punctuation)
stripped = [w.translate(table) for w in annotation_tokens]
# remove remaining tokens that are not alphabetic
words = [word for word in stripped if word.isalpha()]
# filter out stop words
words = [w for w in words if not w in stop_words]
print(words[:100])

['hermione', 'providing', 'aid', 'king', 'husband', 'almost', 'used', 'accessory']


In [34]:
annotations_df["annotation_text"]

0      [hermione, providing, aid, king, husband, almo...
1      Everyone is playing, but Leontes is the only o...
2      I thought it was strange how Leontes asks Mami...
3      It's the giving of hands that leads to the pla...
4      I think this line is very important. Leontes i...
                             ...                        
893    This reminds me of Trump claiming "falsehoods"...
894    I can't read the word "goodly" without also se...
895    This reminds me of Trump claiming "falsehoods"...
896                                             The moon
897    Although the two kings are bff's, the play set...
Name: annotation_text, Length: 898, dtype: object

In [25]:
quotes = annotations_df["annotated_quote"]

In [30]:
for quote in quotes:
    print(quote)
    quote.replace("\r", "")
    quote.replace("\n", "")
    quote.replace("\\", "")
    quote.replace("xa0", "")

print(quotes)

Tongue-tied, our queen?
  Speak you.
Go play, boy, play. Thy mother plays, and I 
[235]  Play too,
Mamillius,
  Art thou my boy?
Give me that hand of yours to kiss.
You kill her double
Hastily lead away.
My blemishes
Then, good my lords, bear witness to his oath.
I, that please some, try all—both joy and terror
  Of good and bad, that makes and unfolds error—
[5]  To me or my swift passage that I slide
  O’er sixteen years, and leave the growth untried
  Of that wide gap, since it is in my power
  To o’erthrow law and in one self-born hour
As you had slept between.
I turn my glass and give my scene such growing
  As you had slept between. Leontes leaving,
  Th’ effects of his fond jealousies so grieving
  That he shuts up himself,
I have heard, sir, of such a man, who hath a
  daughter of most rare note. The report of her is
[45]  extended more than can be thought to begin from
  such a cottage.
appearing what we are, have some question with
  the shepherd, from whose simplicity I thin

AttributeError: 'float' object has no attribute 'replace'

In [26]:
# remove all tokens that are not alphabetic
quote_words = [word for word in quotes if word.isalpha()]
print(quote_words[:100])

AttributeError: 'float' object has no attribute 'isalpha'

In [23]:
# split into words
for i in quotes:
    word_tokenize(quotes[i])
print(quotes)

KeyError: 'Tongue-tied, our queen?\n \xa0Speak you.'

In [15]:
annotations_df["annotation_text"][0] = annotations

In [16]:
annotations_df["annotated_quote"][0] = quotes

In [17]:
annotations_df

,annotation_text,annotated_quote,annotation_location,category
0,Hermione providing aid to her King and husband...,"Tongue-tied, our queen?\n Speak you.",Act 1 Scene 2,Gender
1,"Everyone is playing, but Leontes is the only o...","Go play, boy, play. Thy mother plays, and I \n...",Act 1 Scene 2,None
2,I thought it was strange how Leontes asks Mami...,"Mamillius,\n Art thou my boy?",Act 1 Scene 2,Class
3,It's the giving of hands that leads to the pla...,Give me that hand of yours to kiss.,Act 5 Scene 3,Class
4,I think this line is very important. Leontes i...,You kill her double,Act 5 Scene 3,Gender
...,...,...,...,...
893,"This reminds me of Trump claiming ""falsehoods""...",This is mere falsehood,Act 3 Scene 2,None
894,"I can't read the word ""goodly"" without also se...","“She is a goodly lady,”",Act 2 Scene 1,None
895,"This reminds me of Trump claiming ""falsehoods""...",This is mere falsehood.,Act 3 Scene 2,Pandemics
896,The moon,the wat’ry star,Act 1 Scene 2,None


In [18]:
annotations_df.to_csv('ps_dataset1_cleaned_v2.csv',index=False)